In [1]:
import tensorflow as tf
import tensorflow_recommenders as tfrs
import pandas as pd
import numpy as np
import tensorflow_datasets as tfds
from tensorflow import keras
from typing import Dict, Text

2022-10-13 20:14:13.255523: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-13 20:14:13.546860: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-13 20:14:13.670881: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-13 20:14:14.651085: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
ratings = tfds.load("movielens/1m-ratings", split="train")
movies = tfds.load("movielens/1m-movies", split="train")



2022-10-13 20:14:16.513576: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-10-13 20:14:16.513624: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tyo-Katana-GF66-11UC): /proc/driver/nvidia/version does not exist
2022-10-13 20:14:16.516243: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})
ratings

movies = movies.map(lambda x: x["movie_title"])

In [4]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [5]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie = np.unique(np.concatenate(list(movie_titles)))
unique_ids = np.unique(np.concatenate(list(user_ids)))
unique_movie[:10]

array([b'$1,000,000 Duck (1971)', b"'Night Mother (1986)",
       b"'Til There Was You (1997)", b"'burbs, The (1989)",
       b'...And Justice for All (1979)', b'1-900 (1994)',
       b'10 Things I Hate About You (1999)', b'101 Dalmatians (1961)',
       b'101 Dalmatians (1996)', b'12 Angry Men (1957)'], dtype=object)

In [6]:
model1 = tf.keras.Sequential([
  tf.keras.layers.StringLookup(vocabulary=unique_ids, mask_token=None),
  tf.keras.layers.Embedding(len(unique_ids) + 1, 32),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128)
])



In [7]:
model2 = tf.keras.Sequential([
    tf.keras.layers.StringLookup(vocabulary=unique_movie, mask_token=None),
    tf.keras.layers.Embedding(len(unique_movie) + 1, 32),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128)
])

In [8]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(model2),
  ks = (50, 100)
)


In [9]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [10]:
class MovielensModel(tfrs.Model):

  def __init__(self, model1, model2):
    super().__init__()
    self.model2: tf.keras.Model = model2
    self.model1: tf.keras.Model = model1
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.model1(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.model2(features["movie_title"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)

In [11]:
class NoBaseClassMovielensModel(tf.keras.Model):

  def __init__(self, model1, model2):
    super().__init__()
    self.model2: tf.keras.Model = model2
    self.model1: tf.keras.Model = model1
    self.task: tf.keras.layers.Layer = task

  def train_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    with tf.GradientTape() as tape:

      # Loss computation.
      user_embeddings = self.model1(features["user_id"])
      positive_movie_embeddings = self.model2(features["movie_title"])
      loss = self.task(user_embeddings, positive_movie_embeddings)

      # Handle regularization losses as well.
      regularization_loss = sum(self.losses)

      total_loss = loss + regularization_loss

    gradients = tape.gradient(total_loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

  def test_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    # Loss computation.
    user_embeddings = self.model1(features["user_id"])
    positive_movie_embeddings = self.model2(features["movie_title"])
    loss = self.task(user_embeddings, positive_movie_embeddings)

    # Handle regularization losses as well.
    regularization_loss = sum(self.losses)

    total_loss = loss + regularization_loss

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

In [12]:
model_1 = MovielensModel(model1, model2)
model_1.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [13]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [14]:
model_1.fit(cached_train, epochs=100)

Epoch 1/100
10/10 [==============================] - 12s 878ms/step - factorized_top_k/top_50_categorical_accuracy: 0.0172 - factorized_top_k/top_100_categorical_accuracy: 0.0340 - loss: 71301.6122 - regularization_loss: 0.0000e+00 - total_loss: 71301.6122
Epoch 2/100
10/10 [==============================] - 9s 858ms/step - factorized_top_k/top_50_categorical_accuracy: 0.0263 - factorized_top_k/top_100_categorical_accuracy: 0.0529 - loss: 70218.0526 - regularization_loss: 0.0000e+00 - total_loss: 70218.0526
Epoch 3/100
10/10 [==============================] - 9s 881ms/step - factorized_top_k/top_50_categorical_accuracy: 0.0468 - factorized_top_k/top_100_categorical_accuracy: 0.0866 - loss: 68760.8317 - regularization_loss: 0.0000e+00 - total_loss: 68760.8317
Epoch 4/100
10/10 [==============================] - 8s 826ms/step - factorized_top_k/top_50_categorical_accuracy: 0.0852 - factorized_top_k/top_100_categorical_accuracy: 0.1445 - loss: 66725.7315 - regularization_loss: 0.0000e+00 

In [15]:
ratings = tfds.load("movielens/1m-ratings", split="train")

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})

In [16]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [17]:
movie_titles = ratings.batch(1_000_000).map(lambda x: x["movie_title"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [18]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    user_id, movie_title = inputs

    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

In [19]:
task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [20]:
class MovielensRate(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"], features["movie_title"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("user_rating")

    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

In [21]:
model_2 = MovielensRate()
model_2.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [22]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [23]:
model_2.fit(cached_train, epochs=500)

Epoch 1/500
10/10 [==============================] - 3s 29ms/step - root_mean_squared_error: 2.0880 - loss: 4.0360 - regularization_loss: 0.0000e+00 - total_loss: 4.0360
Epoch 2/500
10/10 [==============================] - 0s 11ms/step - root_mean_squared_error: 1.1732 - loss: 1.3688 - regularization_loss: 0.0000e+00 - total_loss: 1.3688
Epoch 3/500
10/10 [==============================] - 0s 11ms/step - root_mean_squared_error: 1.1182 - loss: 1.2517 - regularization_loss: 0.0000e+00 - total_loss: 1.2517
Epoch 4/500
10/10 [==============================] - 0s 10ms/step - root_mean_squared_error: 1.1074 - loss: 1.2289 - regularization_loss: 0.0000e+00 - total_loss: 1.2289
Epoch 5/500
10/10 [==============================] - 0s 10ms/step - root_mean_squared_error: 1.1018 - loss: 1.2164 - regularization_loss: 0.0000e+00 - total_loss: 1.2164
Epoch 6/500
10/10 [==============================] - 0s 11ms/step - root_mean_squared_error: 1.0950 - loss: 1.2012 - regularization_loss: 0.0000e+00 -

In [24]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [25]:
index = tfrs.layers.factorized_top_k.BruteForce(model_1.model1)

index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model_1.model2)))
)
id = 200
_, titles = index(tf.constant([f'{id}']))
print(f"Recommendations for user {id} : {titles[0, :100]}\n")
list_title = list(titles.numpy().astype('str'))
res = [list(item) for item in list_title]
flat_l = flatten_list(res)
test_ratings = {}
test_movie_titles = flat_l
for movie_title in test_movie_titles:
  test_ratings[movie_title] = model_2({
      "user_id": np.array([f"{id}"]),
      "movie_title": np.array([movie_title])
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Recommendations for user 200 : [b'Entrapment (1999)' b'Arachnophobia (1990)'
 b'Austin Powers: The Spy Who Shagged Me (1999)' b'Alive (1993)'
 b'Bicentennial Man (1999)' b'Secret of NIMH, The (1982)'
 b'Lake Placid (1999)' b'Reality Bites (1994)' b'Falling Down (1993)'
 b'eXistenZ (1999)']

Ratings:
Secret of NIMH, The (1982): [[4.074423]]
Alive (1993): [[3.5014448]]
Arachnophobia (1990): [[3.4655151]]
Falling Down (1993): [[3.4167614]]
Austin Powers: The Spy Who Shagged Me (1999): [[2.772702]]
Bicentennial Man (1999): [[2.6410744]]
Entrapment (1999): [[2.445269]]
eXistenZ (1999): [[2.099279]]
Lake Placid (1999): [[2.0853283]]
Reality Bites (1994): [[2.040011]]
